# Tidying Data in Pandas

In this notebook, we will cover various techniques to tidy data using Pandas. Tidy data is crucial for effective data analysis, ensuring that datasets are clean, consistent, and in a format ready for analysis. We will work through several common data wrangling tasks, including handling datetime objects, string manipulation, reshaping data, merging datasets, and more.

# Handling Date Time Data in Pandas

## Introduction
Date and time data are critical components in data analysis, especially when working with time series data or any dataset where the timing of events is important. However, date formats can vary widely depending on geographic location, data entry methods, or file formats, leading to challenges in consistent data processing.

### Common Date Formats
Some common date formats include:
- `YYYY-MM-DD` (e.g., 2021-07-23) - ISO 8601 standard, widely used internationally.
- `MM/DD/YYYY` (e.g., 07/23/2021) - Common in the United States.
- `DD/MM/YYYY` (e.g., 23/07/2021) - Common in the United Kingdom and other countries.

## Handling Mixed Date Formats
When working with datasets that include mixed date formats (e.g., some entries in `YYYY-MM-DD` and others in `DD/MM/YYYY`), it's essential to standardise the dates for consistent analysis.

### Using `format='mixed'` in Pandas
Pandas provides a `format='mixed'` option in the `pd.to_datetime()` function that allows the library to infer the format for each date individually. This is useful when you cannot predict the format or the dataset is highly inconsistent..


In [6]:
import pandas as pd

# Sample data with different date formats
data = {
    'ID': [1, 2, 3, 4],
    'Event_Date': ['2021-07-23', '2021/08/01', '23-07-2021', '01-08-2021']
}

df = pd.DataFrame(data)

# Converting string dates to datetime objects using mixed format
df['Event_Date'] = pd.to_datetime(df['Event_Date'], format='mixed')

# Extracting components
df['Year'] = df['Event_Date'].dt.year
df['Month'] = df['Event_Date'].dt.month
df['Day'] = df['Event_Date'].dt.day

print(df)

   ID Event_Date  Year  Month  Day
0   1 2021-07-23  2021      7   23
1   2 2021-08-01  2021      8    1
2   3 2021-07-23  2021      7   23
3   4 2021-01-08  2021      1    8


## String Splitting and Normalisation

Text data often needs cleaning and normalisation. We'll cover how to split strings into multiple columns and normalise inconsistent text data.

In [19]:
# Sample data with inconsistent formats
data = {
    'Full_Name': ['John Doe', 'Jane Smith', 'john doe', 'JANE SMITH'],
    'Email': ['john.doe@example.com', 'jane.smith@example.com', 'john.doe2@example.com', 'jane.smith2@example.com']
}

df = pd.DataFrame(data)

# Normalize names by lowercasing
df['Full_Name'] = df['Full_Name'].str.lower()

# Splitting full names into first and last names
df[['First_Name', 'Last_Name']] = df['Full_Name'].str.split(' ', expand=True)

print(df)

    Full_Name                    Email First_Name Last_Name
0    john doe     john.doe@example.com       john       doe
1  jane smith   jane.smith@example.com       jane     smith
2    john doe    john.doe2@example.com       john       doe
3  jane smith  jane.smith2@example.com       jane     smith


## Merging DataFrames

Sometimes you need to combine data from multiple DataFrames. We'll explore how to merge data on specific keys.

In [35]:
# Sample data for merging
df1 = pd.DataFrame({
    'ID': [1, 2, 3],
    'Name': ['John', 'Jane', 'Jim']
})

df2 = pd.DataFrame({
    'ID': [1, 2, 4],
    'Age': [28, 34, 29]
})

# Merging DataFrames on 'ID'
merged_df = pd.merge(df1, df2, on='ID', how='left')

print("Merged DataFrame:")
print(merged_df)

Merged DataFrame:
   ID  Name   Age
0   1  John  28.0
1   2  Jane  34.0
2   3   Jim   NaN


## Filtering Data

Filtering data is an essential skill for isolating relevant subsets of data based on specific criteria. We'll cover how to filter rows using conditions.

In [40]:
# Filtering rows where Age is greater than 30
filtered_df = merged_df[merged_df['Age'] > 30]

print("Filtered DataFrame (Age > 30):")
print(filtered_df)

Filtered DataFrame (Age > 30):
   ID  Name   Age
1   2  Jane  34.0


## Applying Functions with `apply` and `map`

Pandas allows you to apply custom functions to your data. We'll use `apply` for row-wise or column-wise operations and `map` for element-wise transformations.

In [47]:
# Sample data
df = pd.DataFrame({
    'Name': ['John', 'Jane', 'Jim'],
    'Score': [85, 90, 88]
})

# Define a function to increase scores by 5%
def increase_score(score):
    return score * 1.05

# Applying the custom function to increase scores
df['Increased_Score'] = df['Score'].apply(increase_score)

print("DataFrame after applying function to increase scores:")
print(df)

# Define a function to map scores to grades based on ranges
def map_grade(score):
    if 90 <= score <= 100:
        return 'A'
    elif 80 <= score < 90:
        return 'B'
    elif 70 <= score < 80:
        return 'C'
    elif 60 <= score < 70:
        return 'D'
    else:
        return 'F'

# Apply the function to map grades
df['Grade (ranged)'] = df['Score'].apply(map_grade)

# Mapping a dictionary to replace values
grade_mapping = {85: 'B', 90: 'A', 88: 'B+'}
df['Grade (mapped)'] = df['Score'].map(grade_mapping)

print("\nDataFrame after mapping grades based on score ranges:")
print(df)

DataFrame after applying function to increase scores:
   Name  Score  Increased_Score
0  John     85            89.25
1  Jane     90            94.50
2   Jim     88            92.40

DataFrame after mapping grades based on score ranges:
   Name  Score  Increased_Score Grade (ranged) Grade (mapped)
0  John     85            89.25              B              B
1  Jane     90            94.50              A              A
2   Jim     88            92.40              B             B+


# Restaurant Data Analysis Challenges

## Standard Challenges:

### 1. Merge the Datasets
**Challenge:** 
- Merge the `user_ratings.csv` dataset with the `restaurants.csv` dataset based on `placeID`.
- Ensure that all columns from both datasets are included in the final DataFrame.

### 2. Tidy the Data and Deal with Missing Values
**Challenge:** 
- Normalise the DateTime to be the `ISO 8601` standard
- Standardize the `city` column to ensure it matches one of the following formats only: "Ciudad Victoria," "Cuernavaca," "Jiutepec," "San Luis Potosi," or "Soledad." 
- Fill in missing `zip` codes using other available information (such as `state` and `city`).
- Handle any other missing values by filling them with the mode for categorical data and the mean for numeric data.

### 3. Calculate Average Ratings
**Challenge:** 
- For each restaurant (`placeID`), calculate the average `food_rating`, `service_rating`, and overall `rating`.
- Create a new DataFrame with these average ratings.

### 4. Identify Top-Rated Restaurants
**Challenge:** 
- Using the average ratings calculated in the previous challenge, identify the top 5 restaurants.
- Display their `placeID`, `name`, `city`, and average ratings.

### 5. Filter Restaurants by Accessibility
**Challenge:** 
- Filter the merged DataFrame to find all restaurants that are fully accessible (`accessibility` column is "completely").
- Display the `placeID`, `name`, and average rating for these restaurants.

### 6. Payment Method Preferences
**Challenge:** 
- Merge the `user_payment.csv` dataset with the `user_ratings.csv` dataset on `userID`.
- Determine which payment method is most frequently used by users who rate restaurants with a `rating` of 4 or higher.

### 7. Restaurant Availability in Different Cities
**Challenge:** 
- Group the merged DataFrame by `city` and count how many restaurants are available in each city.
- Display the `city` and the corresponding restaurant count.

## Bonus Challenges

### Bonus Challenge 1
Using the rating column, categorise each restaurant into different rating categories:

- "Low" for ratings between 1 and 2.
- "Medium" for ratings between 3 and 4.
- "High" for ratings of 5.

### Bonus Challenge 2
Create a new column called `Quality_Score` that combines both `food_rating` and `service_rating` to provide an overall score for each restaurant. The Quality_Score should be calculated as the weighted average of the `food_rating` and `service_rating`, where `food_rating` is given a weight of 0.7 and `service_rating` is given a weight of 0.3. Use the `.apply()` function to compute this new score.